In [97]:
import pandas as pd
from IPython.display import display

In [1]:
ls ../data

census-data/                data_w_descs_and_census.h5
census_data_aggregated.pkl  shape-files/


In [3]:
df = pd.read_hdf(
    '../data/data_w_descs_and_census.h5', 
    'data_w_descs_and_census', 
    start=0,
    stop=2)

In [10]:
with pd.option_context("display.max_rows", 500):
    display(df.head(1).T)

,0
CASE_ENQUIRY_ID,101000958209
OPEN_DT,11/01/2013 09:27:19 AM
TARGET_DT,11/15/2013 09:27:19 AM
CLOSED_DT,11/27/2013 10:15:45 AM
OnTime_Status,OVERDUE
CASE_STATUS,Closed
CLOSURE_REASON,Case Closed Case Resolved
CASE_TITLE,Sign Repair
SUBJECT,Transportation - Traffic Division
REASON,Signs & Signals


## We only care about the user-generated, non-internal issues

In [149]:
df.Source.drop_duplicates()

0          Constituent Call
1        Employee Generated
4           City Worker App
5              Self Service
9      Citizens Connect App
20       Maximo Integration
434                 Twitter
Name: Source, dtype: object

In [151]:
df = df[df.Source.isin(['Constituent Call', 'Self Service', 'Citizens Connect App', 'Twitter'])]
df.shape

(696504, 155)

## Looking at a sample of 2016 <1h issues

In [104]:
df['<1h_completion'] = (df.CLOSED_DT - df.OPEN_DT).apply(lambda x: x / pd.np.timedelta64(1, 's') < 3600)

In [152]:
N = len(df)
cols = ['<1h_completion', 'CLOSURE_REASON', 'TYPE', 'Source', 'OPEN_DT', 'CLOSED_DT']
aa = df[cols].head(N)

In [154]:
aa.head()

,<1h_completion,CLOSURE_REASON,TYPE,Source,OPEN_DT,CLOSED_DT
0,False,Case Closed Case Resolved,Sign Repair,Constituent Call,2013-11-01 09:27:19,2013-11-27 10:15:45
3,False,Case Closed Case Resolved all roads have been...,Request for Snow Plowing,Constituent Call,2013-02-10 03:12:43,2013-02-14 03:40:21
5,False,Case Closed Case Resolved This request has b...,Graffiti Removal,Self Service,2012-07-27 09:43:38,2012-08-03 10:07:20
7,False,Case Closed Case Resolved THIS HAS BEEN COLLE...,Missed Trash/Recycling/Yard Waste/Bulk Item,Constituent Call,2012-03-14 03:54:15,2012-03-15 12:11:38
8,False,Case Closed Case Noted Sorry for the delayed r...,Street Light Outages,Constituent Call,2012-10-03 11:56:43,2013-03-06 10:29:08


In [159]:
cc = aa[aa.OPEN_DT > '2016-01-01'][['<1h_completion', 'TYPE', 'Source']].groupby(['TYPE', '<1h_completion']).count().unstack()
cc.columns = cc.columns.get_level_values(1)
cc.head()

<1h_completion,False,True
TYPE,,
Abandoned Bicycle,684,42
Abandoned Building,142,8
Abandoned Vehicles,3572,190
Aircraft Noise Disturbance,8,1
Alert Boston,17,8


In [200]:
aa[aa.OPEN_DT > '2016-01-01'].shape

(185648, 6)

In [201]:
aa[aa.OPEN_DT > '2016-01-01']['<1h_completion'].sum()

39802

In [202]:
39802 / float(185648)

0.21439498405584761

In [197]:
with pd.option_context("display.max_rows", 500):
    display(pd.DataFrame(cc[True] / (cc[True] + cc[False])).sort(columns=0, ascending=False).dropna())

,0
TYPE,
Call Log,0.995851
OCR Front Desk Interactions,0.967811
Short Measure - Gas,0.944444
Water in Gas - High Priority,0.800000
No Price on Gas/Wrong Price,0.750000
Automotive Noise Disturbance,0.555556
Scanning Overcharge,0.500000
Heat/Fuel Assistance,0.500000
Item Price Missing,0.500000


In [199]:
CHOSEN_TYPE = 'Needle Pickup'

with pd.option_context("display.max_colwidth", 200):
    display(df[df.OPEN_DT > '2016-01-01'][df.OPEN_DT < '2017-02-01'][df.TYPE == CHOSEN_TYPE][['Source', 'TYPE', '<1h_completion', 'CASE_STATUS', 'description', 'CLOSURE_REASON']].head(30))

,Source,TYPE,<1h_completion,CASE_STATUS,description,CLOSURE_REASON
1522,Constituent Call,Needle Pickup,True,Closed,"Needle Quantity: Three | Property Location Type: Private | Description: Constituent states that she has seen 3 needles. Looking at house, there is a walk way, and to the right side of walkway in r...",Case Closed. Closed date : 2016-12-08 13:58:59.143 Case Resolved 2 syringes recovered from 257 East Cottage Street Dorchester
1802,Citizens Connect App,Needle Pickup,False,Closed,NaN,Case Closed. Closed date : 2016-03-25 12:47:47.763 Case Resolved Recovered and disposed of one needle at 608 mass ave.
2099,Constituent Call,Needle Pickup,False,Closed,Needle Quantity: One | Property Location Type: Public | Description: 1 needle by the bus stop on the left hand sign,Case Closed. Closed date : 2016-09-12 13:30:01.617 Case Resolved 1 syringe recovered from 2997 Washington Street Jamaica Plain
2294,Citizens Connect App,Needle Pickup,True,Closed,Needle and plunger,Case Closed. Closed date : 2016-02-01 14:37:24.08 Case Resolved Responded to 41 East Springfield Street South End.� After an extensive search we were unable to locate a syringe.
2403,Citizens Connect App,Needle Pickup,True,Closed,Needle very close to sidewalk at bottom of hill on property with large multi-family Victorian.,Case Closed. Closed date : 2016-03-19 16:17:49.463 Case Resolved 1 syringe recovered from 176 Poplar Street Roslindale. J Canty BPHC
3106,Citizens Connect App,Needle Pickup,False,Closed,On e Lenox street,Case Closed. Closed date : 2016-02-18 16:44:06.18 Case Resolved 8 syringes recovered from the intersection of Millburn Pl and Reed Street Roxbury.�
3419,Citizens Connect App,Needle Pickup,True,Closed,Behind electrical box behind baseball field in front of old Dearborn school - orchard park.,Case Closed. Closed date : 2016-07-27 14:27:16.363 Case Resolved One syringe recovered from the interesection of Woodrow Wilson Ct. and Ambrose st. Roxbury.
3478,Citizens Connect App,Needle Pickup,False,Closed,Needle on sidewalk,Case Closed. Closed date : 2016-12-08 16:15:27.673 Case Resolved 1 syringe recovered from the Intersection Of Farnham St & Hampden St Roxbury
3665,Citizens Connect App,Needle Pickup,True,Closed,Discarded needle behind Blake House,Case Closed. Closed date : 2016-07-18 14:14:33.343 Case Resolved 1 syringe recovered from behind the Blake House 735 Columbia Road Dorchester
4275,Citizens Connect App,Needle Pickup,False,Closed,NaN,Case Closed. Closed date : 2016-09-02 12:42:33.557 Case Resolved After a thorough search of the area we were unable to locate a syringe at 746 Harrison Ave South End.


## Investigating description nulls by source

In [173]:
df['is_description_null'] = df.description.isnull()

/Users/datascientist/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [178]:
df[df.OPEN_DT > '2016-01-01'][['is_description_null', 'Source', 'CASE_STATUS']].groupby(['Source', 'is_description_null']).count().unstack()

CASE_STATUS       
is_description_null        False  True 
Source                                 
Citizens Connect App       23732  55223
Constituent Call           25661  69105
Self Service                2876   8175
Twitter                      425    451

In [179]:
df[['is_description_null', 'Source', 'CASE_STATUS']].groupby(['Source', 'is_description_null']).count().unstack()

CASE_STATUS        
is_description_null        False   True 
Source                                  
Citizens Connect App       54208   98454
Constituent Call          126314  323666
Self Service               28657   64226
Twitter                      495     484

It looks like whether or not Description exists is largely independent of the source.